# Introduction to Image Registration

### Learning Objectives

* Define the registration problem: finding the spatial transformation that aligns images in the presence of noise
* Understand how registration helps

  * Compare multiple modalities
  * Track changes over time
  * ..

* Understand how insights can be obtained from both resampled images and the spatial transform

## Overview

## Exercise 1: 

## Exercise 2: